# Matrix factorization

## SQLAlchemy session creation

In [62]:
import numpy as np
import pandas as pd

from sqlalchemy.orm import Session
from sqlalchemy import create_engine

DB_URL = "sqlite:///../../bleau_info.db"

engine = create_engine(DB_URL, echo=False)

session = Session(engine)

In [63]:
import sys

sys.path.append('../../')

## Database query

In [64]:
from sqlalchemy import select
from models.repetition import Repetition


ascents = session.execute(
    select(Repetition.user_id, Repetition.boulder_id)
).all()
ascents_df = pd.DataFrame(data=ascents, columns=["user_id", "boulder_id"])
ascents_df

,user_id,boulder_id
0,3,2
1,4,3
2,5,3
3,6,3
4,7,3
...,...,...
520443,68,40233
520444,1321,40233
520445,1321,40234
520446,68,40235


In [65]:
user_boulder_matrix = ascents_df.pivot_table(
    index="user_id",
    columns="boulder_id",
    aggfunc="size",
    fill_value=0,
    dropna=True,
)

user_ids = user_boulder_matrix.index
boulder_ids = user_boulder_matrix.columns

In [66]:
display(user_boulder_matrix)

boulder_id,2,3,5,7,10,11,13,14,15,16,...,40226,40227,40228,40229,40230,40231,40232,40233,40234,40235
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
from scipy.sparse import coo_matrix

user_boulder_coo = coo_matrix(user_boulder_matrix)
print(user_boulder_coo)

<COOrdinate sparse matrix of dtype 'int64'
	with 520448 stored elements and shape (6853, 27665)>
  Coords	Values
  (0, 96)	1
  (0, 149)	1
  (0, 3463)	1
  (0, 6647)	1
  (0, 9307)	1
  (0, 9332)	1
  (0, 9345)	1
  (0, 9359)	1
  (0, 9364)	1
  (0, 9379)	1
  (0, 9385)	1
  (0, 9403)	1
  (0, 9415)	1
  (0, 9417)	1
  (0, 9428)	1
  (0, 10194)	1
  (0, 10197)	1
  (0, 10201)	1
  (0, 10202)	1
  (0, 10203)	1
  (0, 10205)	1
  (0, 10215)	1
  (0, 10216)	1
  (0, 10218)	1
  (0, 10591)	1
  :	:
  (6842, 27575)	1
  (6842, 27581)	1
  (6842, 27583)	1
  (6842, 27584)	1
  (6843, 27465)	1
  (6843, 27487)	1
  (6843, 27527)	1
  (6843, 27528)	1
  (6843, 27534)	1
  (6844, 27471)	1
  (6845, 27489)	1
  (6845, 27499)	1
  (6846, 27496)	1
  (6847, 27509)	1
  (6848, 27533)	1
  (6848, 27550)	1
  (6849, 27558)	1
  (6849, 27574)	1
  (6849, 27585)	1
  (6849, 27586)	1
  (6849, 27588)	1
  (6849, 27590)	1
  (6850, 27558)	1
  (6851, 27607)	1
  (6852, 27640)	1


In [ ]:
new_rows = user_ids[user_boulder_coo.row]
new_cols = boulder_ids[user_boulder_coo.col]
data = user_boulder_coo.data
shape = (max(user_ids) + 1, max(boulder_ids) + 1)

user_boulder_coo = coo_matrix((data, (new_rows, new_cols)))

In [180]:
from implicit.cpu.als import AlternatingLeastSquares

user_boulder_csr = user_boulder_coo.tocsr()
model = AlternatingLeastSquares(
    factors=50, regularization=0.01, alpha=1, iterations=20, random_state=1
)
model.fit(user_items=user_boulder_csr)

100%|██████████| 20/20 [00:01<00:00, 13.95it/s]


In [181]:
from models.boulder import Boulder


item_list = 6735

ids = model.similar_items(itemid=item_list, filter_items=item_list, N=20)[
    0
].tolist()

boulders = session.execute(
    select(Boulder.name, Boulder.id).filter(Boulder.id.in_(ids))
).all()

boulder_dict = {
    boulder_id: boulder_name for boulder_name, boulder_id in boulders
}
boulder_names = [(boulder_dict[boulder_id], boulder_id) for boulder_id in ids]
display(boulder_names)

[('Ouroboros', 18099),
 ('Realist', 22742),
 ('Narcotic', 15798),
 ('La Guerre du Feu (assis)', 2826),
 ('Mundaka', 37745),
 ('La Mesure du Possible', 7822),
 ('Contre Sens', 28315),
 ('Taipan', 6202),
 ('Magneton', 25502),
 ('Le Pilier du Désert (du bas)', 3875),
 ('Le Maître est Talon (assis)', 17857),
 ('pH 12', 4622),
 ('Mastodon (assis)', 21693),
 ('La Directe du Surplomb de la Mée (assis)', 29622),
 ('Bleu Sacré', 11647),
 ('Miss Tick', 3823),
 ('The Maxx', 24485),
 ('Kaïken', 25204),
 ('Tostaky (assis)', 20782),
 ('Big Dragon (assis)', 18803)]